In [ ]:
%load_ext autoreload

import sys
import os
import pickle as pk

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

from pathlib import Path
path = Path("../spqubolib/").resolve().as_posix()
sys.path.append(path)
if True:
    from spqubolib.qmodel.spatial_qmodel_dense import spatial_qmodel_dense
    from spqubolib.qubo import qubo
    from spqubolib.solver import MA_solver

from placement_experiment import get_J_and_h, timer
from placement_plot import plot_runtime, plot_answer, plot_problem, composed_plot_problem_and_answer

In [ ]:
flag_plot = True
flag_run = False
flag_comptime = False
result_path = "run"
comptime_path = "comptime"
image_path = "images"

STAGE = os.environ.get("STAGE", "")
print("STAGE ==", STAGE)
if STAGE=="run":
    flag_plot = False
    flag_run = True
    flag_comptime = False
    os.makedirs(result_path, exist_ok=True)
elif STAGE=="comptime":
    flag_plot = False
    flag_run = False
    flag_comptime = True
    os.makedirs(comptime_path, exist_ok=True)
elif STAGE=="plot":
    flag_plot = True
    flag_run = False
    flag_comptime = False
    os.makedirs(image_path, exist_ok=True)

In [ ]:
#### parameters ####

# problems

W, H = 5, 5 # [length]
r = 0.5 # [length]                 2R = 0.5

coef_line = 10                   # c^(line) = 10
lengthscale_line = 0.05 # [length] σ^(line) = 0.05
lengthscale_blob = 0.3 # [length]  σ^(blob) = 0.3
K_blob = 1000 #                    N^(blob) = 1000

coef = 1
density_ref = 50 / (W * H) # [number/area] K = 2
pmin, pmax = 0, 1.5 #              c^(max) = 1.5
const_placement_cost = pmin # [utility / number]
coef_placement_cost = pmax - pmin # [utility / number]

num_division_max = 40

# experiment 1:

K_long = 10000 # T = 10000
seed = 0

# experiment 2:

K = 1000
cutoff = {
    "fourier": 40, # for fourier, B = 1, ..., 40
    "naive": 13 # for naive , B = 1, ..., 13
}
num_trial = 25

In [ ]:
n_list = list(np.arange(1, num_division_max + 1, 1))
skip_list = [1/n for n in n_list]

In [ ]:
%autoreload

if flag_run:
    sizes = {}
    qmodels = {}
    params = {}
    for n, skip in zip(n_list, skip_list):
        skip_y, skip_x = skip, skip
        np.random.seed(0)
        Wy, Wx, Ry, Rx, J_sp, h, placement_cost = \
            get_J_and_h(H, W,
            skip_y, skip_x, 
            coef_line,
            lengthscale_line, 
            lengthscale_blob, 
            K_blob,
            r, 
            coef, 
            density_ref, 
            coef_placement_cost, 
            const_placement_cost)

        print("Wy: ", Wy, "Wx:", Wx, "Ry:", Ry, "Rx:", Rx)

        xi = np.ones_like(h)

        qf = spatial_qmodel_dense(
            Wy, Wx, Ry, Rx,
            J_sp.astype("f"), 
            h.astype("f"), 
            xi.astype("f"), 
            const=0, 
            mode="fourier"
        )

        qmodels[n] = qf
        sizes[n] = Wy * Wx
        params[n] = (Wy, Wx, skip_y, skip_x, placement_cost)

    with open(f"{result_path}/problems_fixed_lines.pk", "wb") as f:
        pk.dump({
                "qmodels": qmodels, 
                "sizes": sizes,
                "params": params,
                "n_list": n_list
        }, file=f)

In [ ]:
with open(f"{result_path}/problems_fixed_lines.pk", "rb") as f:
    data = pk.load(f)
    qmodels = data["qmodels"]
    sizes = data["sizes"]
    params = data["params"]
    n_list = n_list
settings_largest = params[n_list[-1]]


In [ ]:
%autoreload

def run(mode, q, **kwargs):
    q.set_mode(mode)

    prob = qubo(q, mode="max", vartype="binary")

    solver = MA_solver(
        **kwargs
    )

    # solve and measure the time
    with timer() as t:
        spins, E = prob.solve(solver)
    return t.t

In [ ]:
%autoreload

# solve the largest problem with the fourier version
if flag_run:
    def softplus(x):
        return np.log(1 + np.exp(x))

    def generate_T(K, log_initial_temperature, log_initial_temperature_decrease_relative, param_final_temperature_relative):
        loge = np.log(1e-1) # theta_1 = 0.1 / 100 = 0.001, theta_2 = 0.1 * 1 = 0.1
        # -T'(1) = T0/r = T0 * exp(-logr)
        #   = T0 * exp(log_initial_temperature_decrease_relative)
        logr = -log_initial_temperature_decrease_relative
        param_b = param_final_temperature_relative
        logT0 = log_initial_temperature
        logK = np.log(K)

        logb = param_b * (loge + logr)
        logc = logb - (logK + logr)
        b = np.exp(logb)
        T0 = np.exp(logT0)

        k = 1
        while True:
            if k == 1:
                yield T0
            else:
                yield T0 * b / (b + softplus(logc + np.log(k-1)))
            k = k + 1

    def generate_p(K, p_initial, p_decrease_timescale_relative):
        p_decrease_timescale = K * p_decrease_timescale_relative
        k = 1
        while True:
            yield np.maximum(0, p_initial * (1 - k / p_decrease_timescale))
            k = k + 1

    def generate_c(K, c_increase_timescale_relative):
        k = 1
        c_increase_timescale = K * c_increase_timescale_relative
        while True:
            yield 1
            k = k + 1

    #### parameter after random search (20250417) #### 
    initial_temperature = 0.1 # T_0 = 0.1
    initial_temperature_decrease_relative = 100 # theta_1 = 0.1 / 100 = 0.001
    param_final_temperature_relative = 1 # theta_2 = 0.1 * 1 = 0.1
    p_initial = 0.05 # p_0 = 0.05
    p_decrease_timescale_relative = 1 
    c_increase_timescale_relative = 1 # not working, because c = 1 is fixed

    log_initial_temperature = np.log(initial_temperature)
    log_initial_temperature_decrease_relative = \
        np.log(initial_temperature_decrease_relative)

    run("fourier", qmodels[n_list[-1]], 
        filename_log=f"{result_path}/MA_log_20250417.npz",
        save_ignore=["I_log"],
        save_skip = K_long - 1,
        K=K_long,
        T_generator = generate_T(K_long, 
                                    log_initial_temperature, 
                                    log_initial_temperature_decrease_relative, 
                                    param_final_temperature_relative),

        p_generator = generate_p(K_long, p_initial, p_decrease_timescale_relative),
        c_generator = generate_c(K_long, c_increase_timescale_relative),
    )


In [ ]:
%autoreload

# compare computational time fourier and naive version of spQUBO
if flag_comptime:
    ret = []

    for mode in ["fourier", "naive"]:
        for n in n_list:
            q = qmodels[n]
            print("n, cutoff:", n, cutoff[mode])
            if n < cutoff[mode]:
                for i in range(num_trial):
                    print(f"#### trial {i} ({mode}, n={n}) ####")
                    # run MA with default parameter, because we are only interested in the time rather than solution here
                    t = run(mode, q, verbose=False) 
                    ret.append((i, n, sizes[n], mode, t))

    ret = pd.DataFrame(ret, columns=["trial", "n","size", "mode", "time"])
    with open(f"{comptime_path}/comp_time_20250417.pk", "wb") as f:
        pk.dump(ret, file=f)


**memo**
It takes 65 mins for `comptime` with
```
K = 1000
cutoff = {
    "fourier": 40,
    "naive": 11
}
num_trial = 25
```

-> Experiment size is increaset to `"naive": 13`

# Plot

## Plot runtime

In [ ]:
%autoreload

if flag_plot:
    with open(f"{comptime_path}/comp_time_20250417.pk", "rb") as f:
        ret = pk.load(f)
    plot_runtime(ret, f"{image_path}/runtime_20230611.png")
    plot_runtime(ret, f"{image_path}/runtime_20230611.pdf")


## Plot answer

In [ ]:
%autoreload

if flag_plot:
    data = np.load(f"{result_path}/MA_log_20250417.npz")
    s_log = (data["s_log"] + 1 ) / 2

    plot_answer(s_log[-1, :], settings_largest, f"{image_path}/placement_final.png")


## Plot problem

In [ ]:
q = qmodels[n_list[-1]]

In [ ]:
print("problem size:", q.Lx, q.Ly)
display(pd.DataFrame(q.J.ravel()).quantile([0.01,0.05,0.95,0.99]))
display(pd.DataFrame(q.get_f_J().ravel()).quantile([0.01,0.05,0.95,0.99]))

In [ ]:
%autoreload

if flag_plot:
    plot_problem(q, f"{image_path}/mapping.png")

## Composed plot

In [ ]:
%autoreload

if flag_plot:
    composed_plot_problem_and_answer(q, s_log[-1, :], settings_largest, f"{image_path}/composed_plot.png")